In [ ]:
import sqlalchemy as sql
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from IPython.display import clear_output
import networkx as nx

In [ ]:
engine = sql.create_engine('sqlite:///charters.db', echo=False)
Base = declarative_base()
    

class Charter(Base):
    __tablename__ = 'charters'

    id = sql.Column(sql.String, primary_key=True)
    description = sql.Column(sql.String)
    sawyer = sql.Column(sql.Integer)
    birch = sql.Column(sql.Integer)
    kemble = sql.Column(sql.Integer)
    british_academy = sql.Column(sql.String)
    source_used = sql.Column(sql.String)
    archive = sql.Column(sql.String)
    language = sql.Column(sql.String)
    date = sql.Column(sql.Integer)
    scholarly_date = sql.Column(sql.String)
    scholarly_date_low = sql.Column(sql.Integer)
    scholarly_date_high = sql.Column(sql.Integer)
    scholarly_date_avg = sql.Column(sql.Float)
    text = sql.Column(sql.Text)
    notes = sql.Column(sql.Text)
    asc_source = sql.Column(sql.String)
    pase_source = sql.Column(sql.String)
    pase_witnesses = sql.Column(sql.String)
    
    witnesses = sql.orm.relationship('Person', secondary='charter_witnesses', back_populates='charters')
    
    @property
    def record(self):
        """Gets entry data in dictionary format."""
        return {
            'id': self.id,
            'description': self.description,
            'sawyer': self.sawyer,
            'birch': self.birch,
            'kemble': self.kemble,
            'british_academy': self.british_academy,
            'source_used': self.source_used,
            'archive': self.archive,
            'language': self.language,
            'date': self.date,
            'scholarly_date': self.scholarly_date,
            'scholarly_date_low': self.scholarly_date_low,
            'scholarly_date_high': self.scholarly_date_high,
            'scholarly_date_avg': self.scholarly_date_avg,
            'text': self.text,
            'notes': self.notes,
            'asc_source': self.asc_source,
            'pase_source': self.pase_source,
            'pase_witnesses': self.pase_witnesses
        }
    
    @property
    def coappearances(self):
        """Gives a list of person -> person relationships formed through coappearance on the charter."""
        witnesses = []
        for witness_index, witness in enumerate(self.witnesses):
            if witness_index < len(self.witnesses):
                for cowitness_index, cowitness in enumerate(self.witnesses[witness_index + 1:]):
                    witnesses.append({'source': witness, 'target': cowitness})
        return witnesses
                    
    
class Person(Base):
    __tablename__ = 'people'
    
    id = sql.Column(sql.String, primary_key=True)
    description = sql.Column(sql.String)
    link = sql.Column(sql.String)
    
    charters = sql.orm.relationship('Charter', secondary='charter_witnesses', back_populates='witnesses')
    
    @property
    def record(self):
        """Gets entry data in dictionary format."""
        return {
            'id': self.id,
            'description': self.description,
            'link': self.link
        }
    
    @property
    def earliest_appearance(self):
        """Returns the date of the earliest charter features said person."""
        earliest_charter = None
        for charter in self.charters:
            if not earliest_charter:
                earliest_charter = charter.scholarly_date_avg
            else:
                if charter.scholarly_date_avg < earliest_charter:
                    earliest_charter = charter.scholarly_date_avg
        return earliest_charter

    
class CharterWitness(Base):
    __tablename__ = 'charter_witnesses'
    charter_id = sql.Column(sql.String, sql.ForeignKey('charters.id'), primary_key=True) 
    person_id = sql.Column(sql.String, sql.ForeignKey('people.id'), primary_key=True)
    role = sql.Column(sql.String)
    link = sql.Column(sql.String)
    
    @property
    def record(self):
        """Gets entry data in dictionary format."""
        return {
            'charter_id': self.charter_id,
            'person_id': self.person_id,
            'role': self.role,
            'link': self.link
        }


print('Database Configured Successfully')

In [ ]:
# first, we need to open a session with the local database
Session = sessionmaker(bind=engine)
session = Session()

# create an empty networkx graph
witness_network = nx.Graph()

print('Building network graph...')
# create an empty list to store ids of people already added
witnesses_added = []
for charter in session.query(Charter):
    for coappearance in charter.coappearances:
        source = coappearance['source'].record
        target = coappearance['target'].record
        # if source or target have not been added to nodes, do so
        if source['id'] not in witnesses_added:
            witness_network.add_node(source['id'], **source)
            witnesses_added.append(source['id'])
        if target['id'] not in witnesses_added:
            witness_network.add_node(target['id'], **target)
            witnesses_added.append(target['id'])
        # add edge data from charters
        edge_record = {
            'charter': charter.id,
            'description': charter.description,
            'sawyer': charter.sawyer,
            'birch': charter.birch,
            'kemble': charter.kemble,
            'british_academy': charter.british_academy,
            'source_used': charter.source_used,
            'archive': charter.archive,
            'language': charter.language,
            'date': charter.scholarly_date,
            'scholarly_date': charter.scholarly_date,
            'scholarly_date_low': charter.scholarly_date_low,
            'scholarly_date_high': charter.scholarly_date_high,
            'scholarly_date_avg': charter.scholarly_date_avg,
            'text': charter.text,
            'notes': charter.notes,
            'asc_source': charter.asc_source,
            'pase_source': charter.pase_source,
            'pase_witnesses': charter.pase_witnesses
        }
        witness_network.add_edge(source['id'], target['id'], **edge_record)

def find_greatest(result_set):
    largest_result = None
    for node_id in result_set:
        if not largest_result:
            largest_result = {'keyword': node_id, 'value': result_set[node_id]}
        # check if current value is larger than 'value' in largest_result, set if so
        elif result_set[node_id] > largest_result['value']:
            largest_result = {'keyword': node_id, 'value': result_set[node_id]}
    return largest_result

print('Drawing graph preview...')
nx.draw_spring(witness_network)

print('Calculating (shortest-path) betweeness centrality...')
betweeness_results = nx.betweenness_centrality(witness_network)
largest_betweeness = find_greatest(betweeness_results)
print('{}: {}'.format(largest_betweeness['keyword'], largest_betweeness['value']))

print('Calculating eigenvector centrality...')
eigenvector_results = nx.eigenvector_centrality(witness_network)
largest_eigenvector = find_greatest(eigenvector_results)
print('{}: {}'.format(largest_eigenvector['keyword'], largest_eigenvector['value']))

print('Calculating closeness centrality...')
closeness_results = nx.closeness_centrality(witness_network)
largest_closeness = find_greatest(closeness_results)
print('{}: {}'.format(largest_closeness['keyword'], largest_closeness['value']))

print('Calculating harmonic centrality...')
harmonic_results = nx.harmonic_centrality(witness_network)
largest_harmonic = find_greatest(harmonic_results)
print('{}: {}'.format(largest_harmonic['keyword'], largest_harmonic['value']))